# C, Ctypes, Cython, Augmented Pure Python, Pure Python:
# Performance analysis of coding in Python

## The Mori


<details>
<summary>Evaluate The ``mandelbrot`` fractal image construction using:</summary>
  - Pure Python  
  - Pure C
  - Cython
    - Calling C
    - Cythonize .pyx
    - Cythonize .py (Augmented Pure Python mode)
  - Measure speed of execution
</details>  
<p></p>


In [97]:
# Ignore the man behind the curtain
from __future__ import print_function, division
%matplotlib notebook
from timeit import default_timer as timer
from PIL import Image
import subprocess
import os, sys
import numpy as np
from matplotlib.pylab import subplots


In [99]:
# Somehow, Cython 'Magic' in Jupyter does not provide a means to handle Augmented Pure Python modules (or at least I could not figure it out)
# The solutions adopted is to separately compile the Augmented Pure Python modules in a subdirectory (using setup.py) and use pyximport
# The Jupyter kernel does not seem to even load the compiled modules and loads the Pure Python code instead!
import pyximport
pyximport.install()
sys.path.insert(0, "./c_from_python/cython_sources")

# Once this is run, you need to restart your Jupyter kernel
subprocess.check_call(['python', 'setup.py', 'build_ext', '--inplace'])

running build_ext
building 'cython_sources.app_decorated_mandel' extension
gcc -pthread -B /opt/conda/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/include -fPIC -I/opt/conda/include/python3.10 -c cython_sources/app_decorated_mandel.cpp -o build/temp.linux-x86_64-3.10/cython_sources/app_decorated_mandel.o
g++ -pthread -B /opt/conda/compiler_compat -shared -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib -Wl,-rpath,/opt/conda/lib -Wl,-rpath-link,/opt/conda/lib -L/opt/conda/lib build/temp.linux-x86_64-3.10/cython_sources/app_decorated_mandel.o -o build/lib.linux-x86_64-3.10/cython_sources/app_decorated_mandel.cpython-310-x86_64-linux-gnu.so
building 'cython_sources.app_mandel' extension
gcc -pthread -B /opt/conda/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/conda/include -fPIC -O2 -isystem /opt/conda/incl

0

## Our mission

Create a fractal image. Hmm, what is an image? We decide to define a simple structure to hold the image: width, height, data-as-pointer

In [53]:
class Img(object):
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.data = bytearray(width*height)

width = 1500
height = 1000
image = Img(width, height)

OK, how do we make a fractal image? We loop over the image, doing a calculation at each pixel location $x$, $y$.
For reasons known to only a select few, we normalize the horizontal $x$ values to be from -2 to 1 and the vertical $y$ values to -1 to 1, and then call a function [`mandel`](https://en.wikipedia.org/wiki/Mandelbrot_set) with these normalized values. $x$, $y$ will become the real, imaginary parts of a complex number and we will do some math on that value. 


## The looping function


In [54]:
def create_fractal(image, iters, func, oneval):
    ''' Call a function for each pixel in the image, where
        -2 < real < 1 over the columns and
        -1 < imag < 1 over the rows
    '''
    pixel_size_x = 3.0 / image.width
    pixel_size_y = 2.0 / image.height
    for y in range(image.height):
        imag = y * pixel_size_y - 1
        yy = y * image.width
        for x in range(image.width):
            real = x * pixel_size_x - 2
            
            ret = func(real, imag, iters, oneval) # <---- HERE is the real work
            if ret < 0:
                return ret
            image.data[yy + x] = oneval[0]
    return 0

In [55]:
# This is the calculating function in python
def mandel(x, y, max_iters, value):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    i = 0
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            value[0] = i
            return 0
    value[0] = max_iters
    return max_iters

In [90]:
# OK, lets try it out. Here is our pure python fractal generator

oneval = bytearray(1)  # this is a kind of pointer in pure python
s = timer()
ret = create_fractal(image, 20, mandel, oneval)
e = timer()
if ret < 0:
    print('bad ret value from creat_fractal')
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, mandel, oneval)
    e = timer()
    runs.append(e - s)
pure_python_t = min(runs)
print('pure python required {:.2f} secs'.format(pure_python_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('python_numpy.png')

pure python required 2.95 secs


In [57]:
fig, ax = subplots(1)
img = Image.open('python_numpy.png')
in_data = np.asarray(im, dtype=np.uint8)
#ax.plot(img.histogram())
ax.imshow(in_data) 
ax.set_title('pure python, {:.2f} ms'.format(pure_python_t*1000));

<IPython.core.display.Javascript object>

C version

In [58]:
# print out the file "mandel.c"
with open('mandel.c', 'rt') as fid:
    print(fid.read())

#include "create_fractal.h"
int mandel(float x, float y, int max_iters, unsigned char * val)
{
    int i = 0;
    float cR = x;
    float cI = y;
    float zR = 0;
    float zI = 0;
    for (i = 0; i < max_iters; i++)
    {
        /* in complex notation, z * z + c */
	float prev_zR = zR;
        zR = zR * zR - zI * zI + cR;
        zI = 2 * prev_zR * zI + cI;
        if ((zR * zR + zI * zI) >= 4)
        {
            *val = i;
            return 0;
        }
    }
    *val = max_iters;
    return 1;
}



Test runner file `create_fractal.c`

In [59]:
# print out "create_fractal.c"
with open('create_fractal.c', 'rt') as fid:
    print(fid.read())

#include "create_fractal.h"

int create_fractal(Img img,  int iters) {
    float pixel_size_x = 3.0 / img.width;
    float pixel_size_y = 2.0 / img.height;
    int x, y, ret=0;
    for (y=0; y < img.height; y++) {
        float imag = y * pixel_size_y - 1;
        int yy = y * img.width;
        for (x=0; x < img.width; x++) {
            float real = x * pixel_size_x - 2;
            unsigned char color;
            ret += mandel(real, imag, iters, &color);
            img.data[yy + x] = color;
        }
    }
    return ret;
}



In [60]:
# We will compile the functions into a shared object
# and time the call to create_fractal, as before
with open('main.c', 'rt') as fid:
    print(fid.read())

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include "create_fractal.h"

#ifdef CLOCK_PROCESS_CPUTIME_ID
// call this function to start a nanosecond-resolution timer
struct timespec timer_start(){
    struct timespec start_time;
    clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &start_time);
    return start_time;
}

// call this function to end a timer, returning nanoseconds elapsed as a long
long timer_end(struct timespec start_time){
    struct timespec end_time;
    clock_gettime(CLOCK_PROCESS_CPUTIME_ID, &end_time);
    long diffInNanos = end_time.tv_nsec - start_time.tv_nsec;
    return diffInNanos;
}
#else
#include <sys\timeb.h> 
#endif

int main(int argc, const char *argv[], const char * env[])
{
    int width = 1500;
    int height = 1000;
    int iters = 20;
    FILE * fid = NULL;
    Img img;
    size_t written;
#ifdef CLOCK_PROCESS_CPUTIME_ID
    struct timespec vartime;
#else
    struct timeb start, stop;
#endif
    long time_elapsed_nanos;
    img.width = width;
  

In [61]:
# Compile a shared object, and then compile the exe. If you are following along in Windows, well, good luck.
if sys.platform == 'win32':
    subprocess.check_call(['cl', '/LD', 'mandel.c', 'create_fractal.c', '-Fecreate_fractal.dll'])
    subprocess.check_call(['cl', 'main.c', 'create_fractal.lib', '-Femain.exe'])
else:
    subprocess.check_call(['gcc', '--shared', '-fPIC', '-O3', 'mandel.c', 'create_fractal.c', 
                         '-olibcreate_fractal.so'])
    subprocess.check_call(['gcc', '-O3', 'main.c', '-L.', '-lcreate_fractal', '-omain']);

In [92]:
environ = os.environ.copy()
environ['LD_LIBRARY_PATH'] = environ.get('LD_LIBRARY_PATH', '') + ':.'
if sys.platform == 'win32':
    exe = 'main.exe'
else:
    exe = './main'

runs = []
for i in range(20):
    s = timer()
    p = subprocess.Popen([exe,], stdout=subprocess.PIPE, stderr=subprocess.PIPE, env=environ)
    stdout, stderr = p.communicate()
    stdout = str(stdout)
    e = timer()
    runs.append(int(stdout.split(' ')[2]))
pure_c = min(runs)
print('Pure python is {:.1f} times slower than pure C'.format(1000.0*pure_python_t/pure_c));

Pure python is 40.4 times slower than pure C


In [63]:
from matplotlib.pylab import imshow, show, figure, subplots
fig, ax = subplots(1,2)
with open('c.raw', 'rb') as fid:
    img = Image.frombytes(data=fid.read(), size=(1500,1000), mode="L")
ax[0].imshow(np.asarray(img, dtype='uint8'))
ax[0].set_title('Pure C, {:d} ms'.format(pure_c))

img = Image.open('python_numpy.png')
ax[1].imshow(np.asarray(img, dtype='uint8'))
ax[1].set_title('Pure Python, {:.0f} ms'.format(1000*pure_python_t));

<IPython.core.display.Javascript object>

Note that we compiled ``libcreate_fractal.so`` which wraps our fast C function as a shared object, so maybe we can call it from Python?

We have heard of four methods to interface C with Python: ctypes, Cython, Augmented Pure Python. Let's try them out.

In [64]:
# We will need this to tell the various systems where to find things
if sys.maxsize < 2 ** 33:
    arch = '32'
else:
    arch = '64'
if sys.platform == 'win32':
    dllname = './create_fractal%s.dll' % arch
    libname = 'create_fractal%s' % arch
else:
    dllname = './libcreate_fractal.so'
    libname = 'create_fractal'

In [65]:
#ctypes
# First all the declarations. Each function and struct must be redefined ...
import ctypes

class CtypesImg(ctypes.Structure):
    _fields_ = [('width', ctypes.c_int),
                ('height', ctypes.c_int),
                ('data', ctypes.POINTER(ctypes.c_uint8)), # HUH?
               ]
    array_cache = {}
    def __init__(self, width, height):
        self.width = width
        self.height = height
        # Create a class type to hold the data.
        # Since this creates a type, cache it for reuse rather
        # than create a new one each time
        if width*height not in self.array_cache:
            self.array_cache[width*height] = ctypes.c_uint8 * (width * height)
        # Note this keeps the img.data alive in the interpreter
        self.data = self.array_cache[width*height]() # !!!!!!

    def asmemoryview(self):
        # There must be a better way, but this code will not
        # be timed, so explicit trumps implicit
        ret = self.array_cache[self.width*self.height]()
        for i in range(self.width * self.height):
            ret[i] = self.data[i]
        return memoryview(ret)

ctypesimg = CtypesImg(width, height)

    
# Load the shared object (DLL in Windows)
cdll = ctypes.cdll.LoadLibrary(dllname)

#Fish the function pointers from the shared object and define the interfaces
create_fractal_ctypes = cdll.create_fractal
create_fractal_ctypes.argtypes = [CtypesImg, ctypes.c_int]

mandel_ctypes = cdll.mandel
mandel_ctypes.argtypes = [ctypes.c_float, ctypes.c_float, ctypes.c_int, 
                          ctypes.POINTER(ctypes.c_uint8)]

In [66]:
# We have "typed" the function, it knows what to expect, so this should error
try:
    create_fractal_ctypes()
except TypeError as e:
    print(str(e))

this function takes at least 2 arguments (0 given)


### Ctypes use

Let's run this, twice. Once to call the C implementation of create_fractal, and again with
the python implementation of [create_fractal](#The-looping-function) which calls the c-mandel function 
1.5 million times

In [67]:
s = timer()
create_fractal_ctypes(ctypesimg, 20)
e = timer()
ctypes_onecall = e - s
print('ctypes calling create_fractal required {:.2f} ms'.format(1000*ctypes_onecall))
im = Image.frombuffer("L", (width, height), ctypesimg.asmemoryview(), 'raw', 'L', 0, 1)
im.save('ctypes_fractal.png')

value = (ctypes.c_uint8*1)()
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, mandel_ctypes, value)
    e = timer()
    runs.append(e - s)
ctypes_mandel_t = min(runs)
print('ctypes calling mandel required {:.2f} ms'.format(1000*ctypes_mandel_t))
im = Image.frombuffer("L", (width, height), ctypesimg.asmemoryview(), 'raw', 'L', 0, 1)
im.save('ctypes_mandel.png')

ctypes calling create_fractal required 28.60 ms
ctypes calling mandel required 1102.09 ms


In [68]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


Cython to calling our already compiled C quickly.

In [69]:
%%cython -3 -a -I. -L. -l $libname --link-args=-Wl,-rpath=.

cdef extern from 'create_fractal.h':
    ctypedef struct Img:
        int width
        int height
        unsigned char * data
    
    int create_fractal(Img img, int iters);
    int mandel(float real, float imag, int max_iters,
               unsigned char * val);
    
def cython_c_create_fractal(pyimg, iters):
    cdef Img cimg
    cdef int citers
    cdef unsigned char[::1] tmp = pyimg.data
    
    citers = iters
    cimg.width = pyimg.width
    cimg.height = pyimg.height
    cimg.data = &tmp[0]
    return create_fractal(cimg, citers)


cpdef int cython_c_mandel(float real, float imag, int max_iters,
                        unsigned char[::1] val):
    return mandel(real, imag, max_iters, &val[0])

### Cython use

Let's run this, twice. Once to call the C implementation of create_fractal, and again with
the python implementation of [create_fractal](#The-looping-function) which calls the c-mandel function 
1.5 million times

In [70]:
# use it, remember we have "image" from the pure python version 

s = timer()
cython_c_create_fractal(image, 20)
e = timer()
cython_onecall = e - s
print('cython onecall required {:.2f} ms'.format(1000*cython_onecall))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_fractal.png')

value = bytearray(1)
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, cython_c_mandel, value)
    e = timer()
    runs.append(e - s)
cython_c_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000*cython_c_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_c_mandel.png')

cython onecall required 75.69 ms
cython many calls required 467.21 ms


### Cython Pure Python

This cythonize the python code and compiles the C code
  - This could provide some marginal speed-up, simply due to differences between Cython and CPython
  - All lines are highlighted

In [71]:
%%cython -3 -a -f
#cythonize the python code
def cython_python_mandel(x, y, max_iters, value):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            value[0] = i
            return 0
    value[0] = max_iters
    return max_iters

In [72]:
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, cython_python_mandel, value)
    e = timer()
    runs.append(e - s)
cython_py_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000 * cython_py_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_py_mandel.png')

cython many calls required 2400.25 ms


### Cython with Static Typing

This cythonize the python code and compiles the C code
  - The function is callable from Python (cpdef) and some variables are C-types
  - The -a creates annotations, which indicates the interaction between C and Python as yellow highlights in a .html file
    It can be seen that some Python-C interations are still present (notably the complex variable which is a python type)

In [73]:
%%cython -a -f
#cythonize the python code
cpdef cython_pyx_mandel(double x, double y, int max_iters, unsigned char [:] value):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    cdef int i = 0
    cdef complex c = complex(x,y)
    cdef complex z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            value[0] = i
            return 0
    value[0] = max_iters
    return max_iters

In [74]:
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, cython_pyx_mandel, value)
    e = timer()
    runs.append(e - s)
cython_pyx_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000*cython_pyx_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_pyx_mandel.png')

cython many calls required 596.96 ms


### Cython Augmented Pure Python without optimizing the Python-C interactions

This cythonize the python code and compiles the C code
  - This could provide the same speed-up as the cython code (.pyx), but clearly leave the module in Pure Python
  - Similar lines should be highlighted as previously (This is pre-generated by setup.py and this loaded manually)
    To note: This simple APP can not provide a method to change the local variables -> See decorated APP next

In [75]:
# We will compile the functions into a shared object
# and time the call to create_fractal, as before
with open('cython_sources/app_mandel.py', 'rt') as fid:
    print(fid.read())


# distutils: language = c++

def app_mandel(x, y, max_iters, value):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    c = complex(x,y)
    z = 0.0j
    for i in range(max_iters):
        z = z*z + c
        if (z.real*z.real + z.imag*z.imag) >= 4:
            value[0] = i
            return 0
    value[0] = max_iters
    return max_iters



In [76]:
# We will compile the functions into a shared object
# and time the call to create_fractal, as before
with open('cython_sources/app_mandel.pxd', 'rt') as fid:
    print(fid.read())


cpdef int app_mandel(double real, double imag, int max_iters, unsigned char * value)



In [77]:
from IPython.display import HTML
HTML(filename='cython_sources/app_mandel.html')

In [78]:
from cython_sources.app_mandel import app_mandel
value = bytearray(1)
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, app_mandel, value)
    e = timer()
    runs.append(e - s)
cython_app_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000 * cython_app_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_app_mandel.png')

cython many calls required 2035.87 ms


### Cython Augmented Pure Python with optimizing the Python-C interactions with decorators

This cythonize the python code and compiles the C code
  - This is still a very readable Python (not all the decorators are needed in fact)
  - Almost no lines show Python-C interactions, this cythonized code is almost Pure C

In [79]:
# We will compile the functions into a shared object
# and time the call to create_fractal, as before
with open('cython_sources/app_decorated_mandel.py', 'rt') as fid:
    print(fid.read())


# distutils: language = c++

import cython


# Little help from the .pxd and override of C-functions by inline C and Pure Python calls
# (depending on compiled state)
@cython.exceptval(check=False)
@cython.cfunc
@cython.inline
@cython.returns(cython.double)
def _norm(z: cython.complex):
    if cython.compiled:
        return _std_norm(z)+0
    else:
        return z.real * z.real + z.imag * z.imag


@cython.cfunc
@cython.inline
@cython.returns(cython.complex)
def _complex_number(x: cython.double, y: cython.double):
    if cython.compiled:
        return _std_complex_number(x, y)
    else:
        return complex(x, y)


@cython.exceptval(check=False)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.ccall
@cython.locals(i=cython.int)
@cython.returns(cython.int)
def app_decorated_mandel(x: cython.double,
                         y: cython.double,
                         max_iters: cython.int,
                         value: cython.p_uchar
                         ):
    """
 

In [80]:
# We will compile the functions into a shared object
# and time the call to create_fractal, as before
with open('cython_sources/app_decorated_mandel.pxd', 'rt') as fid:
    print(fid.read())


# distutils: language = c++

cdef extern from "<complex.h>" namespace "std" :
    cdef double _std_norm "abs" (complex z) nogil

cdef inline complex _std_complex_number(double x, double y):
    return x + y * 1j

#cdef inline double _std_norm(double complex z):
#    return z.real * z.real + z.imag * z.imag



In [81]:
from IPython.display import HTML
HTML(filename='cython_sources/app_decorated_mandel.html')

In [82]:
from cython_sources.app_decorated_mandel import app_decorated_mandel

value = bytearray(1)
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, app_decorated_mandel, value)
    e = timer()
    runs.append(e - s)
cython_app_decorated_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000 * cython_app_decorated_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_app_decorated_mandel.png')

cython many calls required 391.94 ms


### Cython with optimizing the Python-C interactions with decorators

This cythonize the python code and compiles the C code
  - For convenience, we use an inline cython file (.pyx without .pxd)

In [83]:
%%cython -+ -3 -a -f
import cython

# This is needed for Jupyter+Cython, but stays in the .pxd and do not prevent the use/debug of Pure Python .py
cdef extern from "<complex.h>" namespace "std" nogil:
    cdef double _std_norm "abs" (complex z)

cdef inline complex _complex_number(double x, double y):
    return (x) + 1j * (y)

@cython.exceptval(check=False)
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.ccall
@cython.locals(i=cython.int)
@cython.returns(cython.int)
def jy_app_decorated_mandel(x: cython.double,
                         y: cython.double,
                         max_iters: cython.int,
                         value: cython.uchar[:]
                         ):
    """
    Given the real and imaginary parts of a complex number,
    determine if it is a candidate for membership in the Mandelbrot
    set given a fixed number of iterations.
    """
    c: cython.complex = _complex_number(x, y)
    z: cython.complex = _complex_number(0, 0)
    _4: cython.double = 4
    for i in range(max_iters):
        z = z * z + c
        if _std_norm(z) >= _4:
            value[0] = i
            return 0
    value[0] = max_iters
    return max_iters


In [84]:
runs = []
for i in range(20):
    s = timer()
    create_fractal(image, 20, jy_app_decorated_mandel, value)
    e = timer()
    runs.append(e - s)

#from scipy.stats import describe
#print(describe(runs))
cython_opt_pyx_mandel_t = min(runs)
print('cython many calls required {:.2f} ms'.format(1000 * cython_opt_pyx_mandel_t))
im = Image.frombuffer("L", (width, height), image.data, "raw", "L", 0, 1)
im.save('cython_opt_pyx_mandel.png')

cython many calls required 562.02 ms


In [85]:
fig, ax = subplots(4, 2)

img = Image.open('python_numpy.png')
ax[0][0].imshow(np.asarray(img, dtype='uint8'))
ax[0][0].set_title('Pure Python, {:.0f} ms'.format(1000 * pure_python_t))

img = Image.open('cython_py_mandel.png')
ax[0][1].imshow(np.asarray(img, dtype='uint8'))
ax[0][1].set_title('Cython .py, {:.0f} ms'.format(1000 * cython_py_mandel_t));

img = Image.open('cython_pyx_mandel.png')
ax[1][0].imshow(np.asarray(img, dtype='uint8'))
ax[1][0].set_title('Cython .pyx naive, {:.0f} ms'.format(1000 * cython_pyx_mandel_t))

img = Image.open('cython_opt_pyx_mandel.png')
ax[1][1].imshow(np.asarray(img, dtype='uint8'))
ax[1][1].set_title('Cython opt. .pyx, {:.0f} ms'.format(1000 * cython_opt_pyx_mandel_t));

img = Image.open('cython_app_mandel.png')
ax[2][0].imshow(np.asarray(img, dtype='uint8'))
ax[2][0].set_title('Cython APP, {:.0f} ms'.format(1000 * cython_app_mandel_t))

img = Image.open('cython_app_decorated_mandel.png')
ax[2][1].imshow(np.asarray(img, dtype='uint8'))
ax[2][1].set_title('Cython Dec. APP, {:.0f} ms'.format(1000 * cython_app_decorated_mandel_t))

img = Image.open('cython_c_mandel.png')
ax[3][0].imshow(np.asarray(img, dtype='uint8'))
ax[3][0].set_title('Cython (C lib), {:.0f} ms'.format(1000 * cython_c_mandel_t))

img = Image.open('ctypes_mandel.png')
ax[3][1].imshow(np.asarray(img, dtype='uint8'))
ax[3][1].set_title('Python Ctypes, {:.0f} ms'.format(1000 * ctypes_mandel_t))


<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Python Ctypes, 1102 ms')

Now let's try and work out who is the good, who the bad

In [96]:
import pandas as pd
#data1 = [
# ['Python',          '{:13.2f} ms'.format(1000*pure_python), ''],
# ['C     ',          '', '{:8.2f} ms'.format(pure_c)],
#]
#pd.DataFrame(data1, columns=['time in ms', 'CreateFractal in Python', 'CreateFractal in C'])
data = [
 ['Python',            '{:13.2f}'.format(1000*pure_python_t) ],
 ['cython - APP',      '{:13.2f}'.format(1000*cython_app_mandel_t)],
 ['ctypes',            '{:13.2f}'.format(1000*ctypes_mandel_t) ],
 ['cython - .pyx',     '{:13.2f}'.format(1000*cython_pyx_mandel_t)],
 ['cython - opt .pyx', '{:13.2f}'.format(1000*cython_opt_pyx_mandel_t)],
 ['cython - c',        '{:13.2f}'.format(1000*cython_c_mandel_t)],
 ['cython - Dec. APP', '{:13.2f}'.format(1000*cython_app_decorated_mandel_t)],
 ['Pure C',            '{:13.2f}'.format(pure_c) ],
]
pd.DataFrame(data, columns=['Method', '1e5M calls from Python'])

,Method,1e5M calls from Python
0,Python,2946.72
1,cython - APP,2035.87
2,ctypes,1102.09
3,cython - .pyx,596.96
4,cython - opt .pyx,562.02
5,cython - c,467.21
6,cython - Dec. APP,391.94
7,Pure C,73.00
